[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/caer200/ocelot_mlp/blob/main/relaxation.ipynb)

### Step 1: Install Required Packages

Installs required libraries:

`pymatgen`: For working with crystal structures.

`tensorflow==2.12`: Required to load and run the M3GNet-based model.

`m3gnet`: Machine-learned interatomic potential model.

`ase`: For working with atomic structures and simulations.

In [ ]:
!pip install pymatgen

In [ ]:
!pip install tensorflow==2.12

In [ ]:
!pip install --no-deps m3gnet
!pip install ase

### Step 2: Clone the Ocelot MLP Repository

Clones the GitHub repository containing the pretrained M3GNet model and example structures.

In [ ]:
!git clone https://github.com/caer200/ocelot_mlp.git

### Step 3: Load the Relaxation Model


Loads the M3GNet model from JSON.

Initializes a `Relaxer` object using the model for performing structure relaxation.

Uses the BFGS optimizer for relaxation steps.

In [ ]:
from m3gnet.models import Relaxer, Potential
import warnings
import json
import tensorflow as tf
from pymatgen.core import Lattice, Structure

for category in (UserWarning, DeprecationWarning):
    warnings.filterwarnings("ignore", category=category, module="tensorflow")


fname = "ocelot_mlp/m3gnet/m3gnet.json"
with open(fname) as f:
    model_serialized = json.load(f)

model_relaxer = tf.keras.models.model_from_json(model_serialized, custom_objects={})
model_relaxer.load_weights("ocelot_mlp/m3gnet/m3gnet")
relaxer = Relaxer(potential=Potential(model_relaxer),optimizer="BFGS")


### Step 4: Load and Relax a Test Structure


Loads a crystal structure from a CIF file (test.cif).

Runs the relaxation for up to 300 steps.

Stores relaxation results, including final structure and trajectory.

In [ ]:
structure = Structure.from_file("ocelot_mlp/test.cif")
relaxer = Relaxer(potential=Potential(model_relaxer),optimizer="BFGS")
relax_results = relaxer.relax(structure, steps=300, verbose=True)

### Step 5: Extract Final Structure

Retrieves the fully relaxed structure from the results.



In [ ]:
relax_results["final_structure"]

### Step 6: Extract Final Energy

Accesses the energy of the final structure from the relaxation trajectory.

In [ ]:
relax_results["trajectory"].energies[-1]